In [1]:
# emulate login with this website
# https://rmm.jaggaer.com/uowisconsin/erd-client/app/login/

# we only know this website is using jquery
import requests
import json
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin, urlencode
import ipywidgets as widgets
from IPython.display import display, HTML
import re
import base64
import pandas as pd

# login page
url = 'https://rmm.jaggaer.com/uowisconsin/erd-client/app/login/'
uri = urlparse(url)
# get base url
base_url = '{uri.scheme}://{uri.netloc}'.format(uri=uri)
print(f'base url: {base_url}')

# create a session
s = requests.Session()

# get the login page
r = s.get(url)
# parse the login page
soup = BeautifulSoup(r.text, 'html.parser')

base url: https://rmm.jaggaer.com


In [2]:
# Based on the above html, we can see that the form is using POST method
# emulate the form data
# !!! remenber to remove the username and password when uploading to github !!!
username = ''
password = ''
lang = 'en_US'
tz = 'US/Central'

form = {'u': username, 'p': password, 'lang': lang, 'tz': tz}

# submit the form to the login url with the form data
r = s.post(urljoin(base_url, soup.form['action']), data=form)
soup = BeautifulSoup(r.text, 'html.parser')

In [3]:
# check if we are logged in, check in soup
if soup.find('div', {'id': 'loginmessages'}):
    print('Login failed')
    print("current url: ", r.url)
else:
    print('Login success')
    print("current url: ", r.url)

Login success
current url:  https://rmm.jaggaer.com/uowisconsin/erd-client/app/secure/home/


In [4]:
# request the current page
r = s.get(r.url)
soup = BeautifulSoup(r.text, 'html.parser')

In [5]:
# Grep all rows and put them into json
rows = soup.find_all('div', {'class': 'row'})
data = []
# for child with id=tile-ViewMyRequestsMenuItem
filter_data = []

for row in rows:
    row_data = {
        'class': row.get('class'),
        'id': row.get('id'),
        'children': []
    }
    for child in row.find_all(recursive=False):
        child_data = {
            'tag': child.name,
            'attributes': child.attrs,
            'text': child.get_text(strip=True)
        }
        print(child_data)
        # true if the id start with "tile-"
        if child_data['attributes'].get('id') and child_data['attributes']['id'].startswith('tile-'):
            filter_data.append(child_data)
        row_data['children'].append(child_data)
    data.append(row_data)

# Convert to JSON
json_data = json.dumps(data, indent=4)

# Save to a file
output_path = 'main_page.json'
with open(output_path, 'w', encoding='utf-8') as json_file:
    json_file.write(json_data)

print(f'Extracted data has been saved to {output_path}')

# save the filtered data to a file
output_path = 'filtered_data.json'
json_data = json.dumps(filter_data, indent=4)
with open(output_path, 'w', encoding='utf-8') as json_file:
    json_file.write(json_data)

print(f'Filtered data has been saved to {output_path}')

{'tag': 'div', 'attributes': {'class': ['col-sm-11', 'no-gutters']}, 'text': 'Research Material Management'}
{'tag': 'div', 'attributes': {'id': 'quicknav', 'class': ['col-sm-13', 'no-gutters']}, 'text': 'Wickens ChemUWMSNUser PreferencesPreferred Search LocationsAlternate Container LocationsReceiving Search LocationsManage Sub-locationsLogoutVersion\xa024.1.0.423JAGGAER Privacy Policy'}
{'tag': 'div', 'attributes': {'class': ['col-sm-24']}, 'text': ''}
{'tag': 'div', 'attributes': {'id': 'toolbar', 'class': ['col-24'], 'style': 'margin-bottom: 1px;'}, 'text': ''}
{'tag': 'div', 'attributes': {'class': ['col-sm-24', 'col-md-14', 'col-lg-10']}, 'text': 'Container Search (use * to perform a wildcard search)Source SearchMaterial SearchMaterial CreateReceivingStructure SearchView My RequestsScan OperationsContainer Operations WorksheetAssign Users to Principal InvestigatorProjectsContainer Inventory ReconciliationSchedule ReportView Completed ReportsTo Do List'}
{'tag': 'div', 'attributes'

### Source Search tile emulation

In [6]:
# we will first focus on Source Search tile
# the id of the tile is tile-SourceSearchMenuItem
# get the url of the tile under data-url attribute
sourcesearch_url = base_url + [x['attributes']['data-url'] for x in filter_data if x['attributes']['id'] == 'tile-SourceSearchMenuItem'][0]

# change the section of this new url
r = s.get(sourcesearch_url, allow_redirects=True)
soup = BeautifulSoup(r.text, 'html.parser')
print(f'current url: {r.url}')
print(f'current title: {soup.title.string}')

current url: https://rmm.jaggaer.com/uowisconsin/erd-client/app/secure/sourcesearch/
current title: JAGGAER - Research Material Management  Source Search


In [7]:
"""<select id="advancedsearchtype" name="advancedsearchtype" class="form-control">
                    <option value="search-materialidentifiers">Material Identifier</option>
                    <option value="search-catalog">Catalog # / Mfr Part #</option>
                    <option value="search-term-mustinclude">Must Include  </option>
                    <option value="search-term-exact">Exact Phrase</option>
                    <option value="search-term-includeany">Include Any</option>
    </select>
"""

# example payload
"""
csrftoken: 88781371-9faf-4580-8afc-2b9a1af1cf40
searchterm: thianthrene
advancedsearchtype: search-materialidentifiers
advancedsearchterms: 
excludeterms: 
channels: InventoryLocationsCatalog
channels: StoreRoomsCatalog
"""

# grep the form content from the page
searchform_structure = soup.find('form', {'id': 'searchform'})

# grep the csrf token
csrftoken = searchform_structure.find('input', {'name': 'csrftoken'})['value']

form_action_url = urljoin(base_url, searchform_structure['action'])

# Create input widgets for form data
search_term_input = widgets.Text(
    description='Search Term:', 
    value='thianthrene',
    style={'description_width': 'initial'},
    display='flex',
)
advanced_search_type_dropdown = widgets.Dropdown(
    options=[
        ('Material Identifier', 'search-materialidentifiers'),
        ('Catalog # / Mfr Part #', 'search-catalog'),
        ('Must Include', 'search-term-mustinclude'),
        ('Exact Phrase', 'search-term-exact'),
        ('Include Any', 'search-term-includeany')
    ],
    description='Advanced Search Type:',
    value='search-materialidentifiers',
    style={'description_width': 'initial'},
    display='flex',
)
advanced_search_terms_input = widgets.Text(
    description='Advanced Search Terms:', 
    value='',
    style={'description_width': 'initial'},
    display='flex',
)
exclude_terms_input = widgets.Text(
    description='Exclude Terms:', 
    value='',
    style={'description_width': 'initial'},
    display='flex',
)
channel_dropdown = widgets.Dropdown(
    options=[
        ('Inventory Locations Catalog', 'InventoryLocationsCatalog'),
        ('Store Rooms Catalog', 'StoreRoomsCatalog')
    ],
    description='Channel:',
    style={'description_width': 'initial'},
    display='flex',
)

# Cache to store fetched data
cache = {}

In [8]:
# Function to handle form submission and show results
def on_form_submit(searchterm, advancedsearchtype, advancedsearchterms, excludeterms, selected_channel, csrftoken, cache, current_page=1) -> requests.Response:
    form_data = {
        'csrftoken': csrftoken,
        'searchterm': searchterm,
        'advancedsearchtype': advancedsearchtype,
        'advancedsearchterms': advancedsearchterms,
        'excludeterms': excludeterms,
        'channels': [selected_channel]
    }
    cache_key = (form_data['searchterm'], form_data['advancedsearchtype'], form_data['advancedsearchterms'], form_data['excludeterms'], form_data['channels'][0], csrftoken, current_page)

    # check if the data is already in the cache, don't check the total number of pages
    if (cache_key in cache):
        response = cache[cache_key]
        soup = BeautifulSoup(response.text, 'html.parser')

        # Display the current URL and content
        print(f'Current URL: {response.url}')
        print(f'Fetched data from CACHE for search term "{searchterm}", advanced search type "{advancedsearchtype}", channel "{selected_channel}, page {current_page}"')

        display_result(soup, form_data, current_page, detect_total_num_pages(soup))
        return response
    
    response = s.post(form_action_url, data=form_data)
    if response.status_code == 200:
        print(f'Post form data to {form_action_url} successfully')
    else:
        print(f'Failed to post form data to {form_action_url}')
        return response

    # Proceed with the GET request to fetch and display results
    # https://rmm.jaggaer.com/uowisconsin/erd-client/app/secure/sourcesearch/results?p=1&channel=InventoryLocationsCatalog
    result_url = base_url + '/uowisconsin/erd-client/app/secure/sourcesearch/results?' + urlencode({'p': current_page, 'channel': selected_channel})
    
    response = s.get(result_url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Print the current URL and content
        print(f'Current URL: {response.url}')
        # Display the number of pages
        print(f'Fetched data from URL "{response.url}" for search term "{searchterm}", advanced search type "{advancedsearchtype}", channel "{selected_channel}", page {current_page} of {detect_total_num_pages(soup)}')

        # Store the fetched data in the cache
        cache[cache_key] = response

        # pass to display result function
        display_result(soup, form_data, current_page, detect_total_num_pages(soup))
    else:
        print(f'Fail to fetch data from {result_url}, error code: {response.status_code}, error message: {response.text}')

    return response

# a function to detect the total number of pages
def detect_total_num_pages(soup):
    # Detect the total number of pages
    pager = soup.find('nav', {'id': 'pager'})
    if pager:
        page_links = pager.find_all('a', class_='page-link')
        total_num_pages = len(page_links) - 2 if page_links else 1
    else:
        total_num_pages = 1

    return total_num_pages

# Function to display results
def display_result(soup, form_data, current_page, total_num_pages):
    # Extract structureMolFiles from the script tag
    script_tag = soup.find('script', string=re.compile('window\.structureMolFiles'))
    script_content = script_tag.string if script_tag else ''
    structure_mol_files = {}
    for match in re.finditer(r'window\.structureMolFiles\[(\d+)\]\s*=\s*\'(.*?)\';', script_content, re.DOTALL):
        key, value = match.groups()
        structure_mol_files[int(key)] = value.replace('\\n', '\n')
    # Render ChemDraw structures
    render_chemdraw_structures(structure_mol_files)

    # Extract and display search results
    search_results = extract_search_results(soup)
    
    # Save search results to a JSON file
    # custom file name related to the search term, channel, search type, page number
    file_name = f'search_results_{search_term_input.value}_{channel_dropdown.value}_{advanced_search_type_dropdown.value}_page_{current_page}.json'
    with open(file_name, 'w', encoding='utf-8') as json_file:
        json.dump(search_results, json_file, indent=4)
    print(f"Search results have been saved to {file_name}")

    # Display the results in a table
    display_results_table(search_results)

    # Display pagination buttons
    if total_num_pages > 1:
        page_input = widgets.BoundedIntText(
            value=current_page,
            min=1,
            max=total_num_pages,
            description='Page:',
            style={'description_width': 'initial'}
        )
        page_button = widgets.Button(description="Go to Page")
        page_button.on_click(lambda b: on_form_submit(form_data['searchterm'], form_data['advancedsearchtype'], form_data['advancedsearchterms'], form_data['excludeterms'], form_data['channels'][0], csrftoken, cache, page_input.value))

        # go to next page button & go back to previous page button, handle first and last page
        # grey out the button if it is the first or last page
        if current_page == 1:
            prev_page_button = widgets.Button(description="Previous Page", disabled=True)
        else:
            prev_page_button = widgets.Button(description="Previous Page")
            prev_page_button.on_click(lambda b: on_form_submit(form_data['searchterm'], form_data['advancedsearchtype'], form_data['advancedsearchterms'], form_data['excludeterms'], form_data['channels'][0], csrftoken, cache, current_page - 1))

        if current_page == total_num_pages:
            next_page_button = widgets.Button(description="Next Page", disabled=True)
        else:
            next_page_button = widgets.Button(description="Next Page")
            next_page_button.on_click(lambda b: on_form_submit(form_data['searchterm'], form_data['advancedsearchtype'], form_data['advancedsearchterms'], form_data['excludeterms'], form_data['channels'][0], csrftoken, cache, current_page + 1))
        
        display(widgets.HBox([prev_page_button, page_input, page_button, next_page_button]))

# Function to extract all search results
def extract_search_results(soup):
    search_results = []
    col_24_divs = soup.find_all('div', class_='col-24 p-0')

    current_supplier = None
    current_brand = None

    for col_24_div in col_24_divs:
        for div in col_24_div.find_all('div', recursive=False):
            if 'sourcesearch-group' in div.get('class', []):
                current_supplier = div.find('div', class_='sourcesearch-supplier').find('span').text.strip()
                current_brand = div.find('div', class_='sourcesearch-brand').find('span').text.strip()
            elif 'grouped-sourcesearch-subrow' in div.get('class', []):
                search_results.append(extract_data(div, current_supplier, current_brand))

    return search_results

# Function to extract and structure data from a result entry
def extract_data(result, supplier, brand):
    label_name = result.find('div', class_='text-bold', title=True)
    label_name = label_name.text.strip() if label_name else ''
    
    location = result.find('div', title="Location")
    location = location.text.strip() if location else ''
    
    bar_code = result.find('div', title="Bar Code")
    bar_code = bar_code.text.strip() if bar_code else ''
    
    supplier_catalog = result.find('div', title="Supplier Catalog #")
    supplier_catalog = supplier_catalog.text.strip() if supplier_catalog else ''
    
    manufacturer_part = result.find('div', title="Manufacturer/Brand Part #")
    manufacturer_part = manufacturer_part.text.strip() if manufacturer_part else ''
    
    # match exact class name for the created date, which is col-12
    created_date = result.find(lambda tag: tag.name == 'div' and tag.get('class') == ['col-12'])
    created_date = created_date.text.strip() if created_date else ''
    
    package = result.find('div', title="Package")
    package = package.text.strip() if package else ''
    
    purity = result.find('div', title="Purity")
    purity = purity.text.strip() if purity else ''
    
    view_container_href = base_url + result.find('a', title="View Container")['href'] if result.find('a', title="View Container") else ''

    return {
        'Supplier': supplier,
        'Brand': brand,
        'Label Name': label_name,
        'Location': location,
        'Bar Code': bar_code,
        'Supplier Catalog #': supplier_catalog,
        'Manufacturer/Brand Part #': manufacturer_part,
        'Created Date': created_date,
        'Package': package,
        'Purity': purity,
        'View Container URL': view_container_href
    }

# Function to render ChemDraw structures, this function will post a request to the server with the mol_files to get the images in base64 format
def render_chemdraw_structures(structure_mol_files):
    image_url = base_url + '/uowisconsin/erd-client/app/secure/structure/getchemdrawstructures'
    mol_files = list(structure_mol_files.values())
    image_ids = list(structure_mol_files.keys())
    
    if mol_files:
        iters = (len(mol_files) + 24) // 25  # Calculate the number of iterations needed
        for i in range(iters):
            lower_bound = i * 25
            upper_bound = min((i + 1) * 25, len(mol_files))
            chunk = mol_files[lower_bound:upper_bound]
            data = {
                'molfiles[]': chunk,
                'lowerbound': lower_bound,
                'csrftoken': csrftoken
            }
            response = s.post(image_url, data=data)
            if response.status_code == 200:
                data = response.json()
                if data['error'] == 'false':
                    for idx, image in enumerate(data['images']):
                        img_id = f"structure-img-{lower_bound + idx}"
                        display(HTML(f'<img id="{img_id}" src="data:image/svg+xml;base64,{base64.b64encode(image.encode("utf-8")).decode("utf-8")}" width="200px" height="200px" class="bg-white">'))
                else:
                    print("Error getting structure images for mol files")
            else:
                print("Error getting structure images for mol files")

# Function to display results in a tabular format
def display_results_table(data):
    df = pd.DataFrame(data)
    display(df)

In [ ]:
# Button to submit the form
def submit_button_clicked(b):
    on_form_submit(
        search_term_input.value, 
        advanced_search_type_dropdown.value, 
        advanced_search_terms_input.value, 
        exclude_terms_input.value, 
        channel_dropdown.value, 
        csrftoken, 
        cache
    )

submit_button = widgets.Button(description="Submit")
submit_button.on_click(submit_button_clicked)

# Display the input widgets and submit button
display(widgets.VBox([search_term_input, advanced_search_type_dropdown, advanced_search_terms_input, exclude_terms_input, channel_dropdown]))
display(submit_button)

Button(description='Submit', style=ButtonStyle())

Post form data to https://rmm.jaggaer.com/uowisconsin/erd-client/app/secure/sourcesearch/search successfully
Current URL: https://rmm.jaggaer.com/uowisconsin/erd-client/app/secure/sourcesearch/results?p=1&channel=InventoryLocationsCatalog
Fetched data from URL "https://rmm.jaggaer.com/uowisconsin/erd-client/app/secure/sourcesearch/results?p=1&channel=InventoryLocationsCatalog" for search term "acetonitrile", advanced search type "search-materialidentifiers", channel "InventoryLocationsCatalog", page 1 of 5


Search results have been saved to search_results_acetonitrile_InventoryLocationsCatalog_search-materialidentifiers_page_1.json


,Supplier,Brand,Label Name,Location,Bar Code,Supplier Catalog #,Manufacturer/Brand Part #,Created Date,Package,Purity,View Container URL
0,Fisher Scientific Co. (Chem Manager Only),Fisher Bioreagent,"Acetonitrile, -99.9%, C2H3N, CAS Number-75-05-...","UWMSN>Chemistry Bldg-0047>6>6319 (WALDE, R...",MSN9161361,BP11704,BP11704,3/29/2024,4 L (4 L),>= 99.9000%,https://rmm.jaggaer.com/uowisconsin/erd-client...
1,Legacy,Aldrich,Acetonitrile,UWMSN>Chemistry Bldg-0047>6>6369>Dylan (Ch...,MSN0110381,,271004,6/18/2023,1 L (1 L),,https://rmm.jaggaer.com/uowisconsin/erd-client...
2,Legacy,Aldrich,Acetonitrile,UWMSN>Chemistry Bldg-0047>6>6229>Jimin (Ch...,MSN0110383,,271004,6/18/2023,1 L (1 L),,https://rmm.jaggaer.com/uowisconsin/erd-client...
3,Legacy,Aldrich,Acetonitrile,UWMSN>Chemistry Bldg-0047>6>6225>Ted (Chem...,MSN0110384,,271004,6/18/2023,1 L (1 L),,https://rmm.jaggaer.com/uowisconsin/erd-client...
4,Legacy,Aldrich,Acetonitrile,UWMSN>Chemistry Bldg-0047>6>6225>Jennifer>Solv...,MSN0110390,,271004,6/18/2023,1 L (1 L),,https://rmm.jaggaer.com/uowisconsin/erd-client...
...,...,...,...,...,...,...,...,...,...,...,...
95,Legacy,Acros Organics,Acetonitrile,UWMSN>Chemistry Bldg-0047>3>3361>Specialty Che...,MSN9025243,,448391000,7/12/2023,100 ML (100 ML),99.9,https://rmm.jaggaer.com/uowisconsin/erd-client...
96,Legacy,Agilent,acetonitrile optima,UWMSN>Chemistry Bldg-0047>2>2142A>Cabinet>Shel...,MSN9031474,,G2421A,8/1/2023,100 ML (100 ML),,https://rmm.jaggaer.com/uowisconsin/erd-client...
97,Legacy,Aldrich,"Acetonitrile, HPLC",UWMSN>Chemistry Bldg-0047>3>3309B>Solvent cabi...,MSN9002379,,34851,10/3/2022,4 L (4 L),,https://rmm.jaggaer.com/uowisconsin/erd-client...
98,Legacy,Aldrich,"Acetonitrile, Anhydrous",UWMSN>Chemistry Bldg-0047>3>3309B>Solvent cabi...,MSN9077517,,271004,10/3/2022,1 L (1 L),,https://rmm.jaggaer.com/uowisconsin/erd-client...


Post form data to https://rmm.jaggaer.com/uowisconsin/erd-client/app/secure/sourcesearch/search successfully
Current URL: https://rmm.jaggaer.com/uowisconsin/erd-client/app/secure/sourcesearch/results?p=2&channel=InventoryLocationsCatalog
Fetched data from URL "https://rmm.jaggaer.com/uowisconsin/erd-client/app/secure/sourcesearch/results?p=2&channel=InventoryLocationsCatalog" for search term "acetonitrile", advanced search type "search-materialidentifiers", channel "InventoryLocationsCatalog", page 2 of 5


...

Search results have been saved to search_results_acetonitrile_InventoryLocationsCatalog_search-materialidentifiers_page_3.json


,Supplier,Brand,Label Name,Location,Bar Code,Supplier Catalog #,Manufacturer/Brand Part #,Created Date,Package,Purity,View Container URL
0,Legacy,Sigma-Aldrich,acetonitrile,UWMSN>Chemistry Bldg-0047>3>3361>Flammables ...,MSN0126871,,34851,7/12/2023,4 L (4 L),99.9,https://rmm.jaggaer.com/uowisconsin/erd-client...
1,Legacy,Sigma-Aldrich,acetonitrile,UWMSN>Chemistry Bldg-0047>3>3361>Flammables ...,MSN0126892,,34851,7/12/2023,4 L (4 L),99.9,https://rmm.jaggaer.com/uowisconsin/erd-client...
2,Legacy,Sigma-Aldrich,acetonitrile,UWMSN>Chemistry Bldg-0047>3>3201>Flammables Ca...,MSN0117817,,34851,7/14/2023,4 L (4 L),? 99.9%,https://rmm.jaggaer.com/uowisconsin/erd-client...
3,Legacy,Sigma-Aldrich,Acetonitrile,UWMSN>Chemistry Bldg-0047>4>4201>Flammables ...,MSN9076405,,34851,7/19/2023,4 L (4 L),,https://rmm.jaggaer.com/uowisconsin/erd-client...
4,Legacy,Sigma-Aldrich,Acetonitrile,UWMSN>Chemistry Bldg-0047>4>4201>Bench 2 (...,MSN9076526,,34851,7/19/2023,4 L (4 L),,https://rmm.jaggaer.com/uowisconsin/erd-client...
...,...,...,...,...,...,...,...,...,...,...,...
95,Sigma Aldrich Inc (Chem Manager Only),SIGMA-ALDRICH,Acetonitrile,"UWMSN>Chemistry Bldg-0047>7>7325 (Chem, Za...",MSN9064446,34851-4X4L,34851,11/8/2023,4 L (4 L),>= 99.9000%,https://rmm.jaggaer.com/uowisconsin/erd-client...
96,Sigma Aldrich Inc (Chem Manager Only),SIGMA-ALDRICH,Acetonitrile,"UWMSN>Chemistry Bldg-0047>7>7325 (Chem, Za...",MSN9064448,34851-4X4L,34851,11/8/2023,4 L (4 L),>= 99.9000%,https://rmm.jaggaer.com/uowisconsin/erd-client...
97,Sigma Aldrich Inc (Chem Manager Only),SIGMA-ALDRICH,Acetonitrile,UWMSN>Chemistry Bldg-0047>8>8114/8118 (Che...,MSN9064310,34851-4X4L,34851,11/9/2023,4 L (4 L),>= 99.9000%,https://rmm.jaggaer.com/uowisconsin/erd-client...
98,Sigma Aldrich Inc (Chem Manager Only),SIGMA-ALDRICH,Acetonitrile,UWMSN>Chemistry Bldg-0047>7>7120/7124 (Che...,MSN9064311,34851-4X4L,34851,11/9/2023,4 L (4 L),>= 99.9000%,https://rmm.jaggaer.com/uowisconsin/erd-client...


Post form data to https://rmm.jaggaer.com/uowisconsin/erd-client/app/secure/sourcesearch/search successfully
Current URL: https://rmm.jaggaer.com/uowisconsin/erd-client/app/secure/sourcesearch/results?p=4&channel=InventoryLocationsCatalog
Fetched data from URL "https://rmm.jaggaer.com/uowisconsin/erd-client/app/secure/sourcesearch/results?p=4&channel=InventoryLocationsCatalog" for search term "acetonitrile", advanced search type "search-materialidentifiers", channel "InventoryLocationsCatalog", page 4 of 5


Search results have been saved to search_results_acetonitrile_InventoryLocationsCatalog_search-materialidentifiers_page_4.json


,Supplier,Brand,Label Name,Location,Bar Code,Supplier Catalog #,Manufacturer/Brand Part #,Created Date,Package,Purity,View Container URL
0,Sigma Aldrich Inc (Chem Manager Only),SIGMA-ALDRICH,Acetonitrile,UWMSN>Chemistry Bldg-0047>7>7120/7124 (Che...,MSN9064314,34851-4X4L,34851,11/9/2023,4 L (4 L),>= 99.9000%,https://rmm.jaggaer.com/uowisconsin/erd-client...
1,Sigma Aldrich Inc (Chem Manager Only),SIGMA-ALDRICH,Acetonitrile,UWMSN>Chemistry Bldg-0047>5>5114/5118 (Che...,MSN9064319,34851-4X4L,34851,11/9/2023,4 L (4 L),>= 99.9000%,https://rmm.jaggaer.com/uowisconsin/erd-client...
2,Sigma Aldrich Inc (Chem Manager Only),SIGMA-ALDRICH,Acetonitrile,UWMSN>Chemistry Bldg-0047>5>5114/5118 (Che...,MSN9032085,34851-4X4L,34851,12/13/2023,4 L (4 L),>= 99.9000%,https://rmm.jaggaer.com/uowisconsin/erd-client...
3,Sigma Aldrich Inc (Chem Manager Only),SIGMA-ALDRICH,Acetonitrile,UWMSN>Chemistry Bldg-0047>8>8123/8127 (Che...,MSN9032090,34851-4X4L,34851,12/13/2023,4 L (4 L),>= 99.9000%,https://rmm.jaggaer.com/uowisconsin/erd-client...
4,Sigma Aldrich Inc (Chem Manager Only),SIGMA-ALDRICH,Acetonitrile,UWMSN>Chemistry Bldg-0047>8>8114/8118 (Che...,MSN9032093,34851-4X4L,34851,12/13/2023,4 L (4 L),>= 99.9000%,https://rmm.jaggaer.com/uowisconsin/erd-client...
...,...,...,...,...,...,...,...,...,...,...,...
95,Sigma Aldrich Inc (Chem Manager Only),SIGMA-ALDRICH,Acetonitrile,UWMSN>Chemistry Bldg-0047>7>7325>Flammables ...,MSN0138441,34851-4X4L,34851,6/4/2024,4 L (4 L),>= 99.9000%,https://rmm.jaggaer.com/uowisconsin/erd-client...
96,Sigma Aldrich Inc (Chem Manager Only),SIGMA-ALDRICH,Acetonitrile,"UWMSN>Chemistry Bldg-0047>3>3201 (Chem, Ch...",MSN0138442,34851-4X4L,34851,6/4/2024,4 L (4 L),>= 99.9000%,https://rmm.jaggaer.com/uowisconsin/erd-client...
97,Sigma Aldrich Inc (Chem Manager Only),SIGMA-ALDRICH,Acetonitrile,UWMSN>Chemistry Bldg-0047>7>7325>Flammables ...,MSN0138444,34851-4X4L,34851,6/4/2024,4 L (4 L),>= 99.9000%,https://rmm.jaggaer.com/uowisconsin/erd-client...
98,Sigma Aldrich Inc (Chem Manager Only),SIGMA-ALDRICH,Acetonitrile,"UWMSN>Chemistry Bldg-0047>5>5222 (ARNDT, A...",MSN0138445,34851-4X4L,34851,6/4/2024,4 L (4 L),>= 99.9000%,https://rmm.jaggaer.com/uowisconsin/erd-client...
